In [ ]:
import numpy as np
import pandas as pd

import nltk, re
from nltk.corpus import stopwords
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
texts_month = pd.read_csv('data/news/texts_31days.csv', index_col=0)
texts_first = pd.read_csv('data/news/texts_12.04-13.04.csv', index_col=0)
texts_second = pd.read_csv('data/news/texts_13.04-14.04.csv', index_col=0)
texts = pd.concat([texts_month, texts_first, texts_second])
texts.index.names = ['url_id']

In [ ]:
# Using texts.csv to make urls for each url_id
def make_urls_df():
    texts = pd.read_csv('data/news/texts.csv')

    tag_cleaned = texts['tag'].str.split().str.get(0)
    texts['tag_cleaned'] = tag_cleaned
    texts['url_id'] = texts['url_id'].astype(str)
    texts['pagePath'] = '/t/' + texts['tag_cleaned'] + '/' + texts['url_id']

    urls = texts.drop(['subtitle', 'tag', 'tag_cleaned'], axis=1)
    return urls

urls = make_urls_df()
urls.dropna(how='any', inplace=True)
urls.drop_duplicates(['title'], inplace=True)

In [ ]:
# Text cleaning
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("russian"))
        tokens = [w for w in tokens if not w in stops]
    
#     text = " ".join(tokens)
    return tokens

# PoS tagging
from ufal.udpipe import Model, Pipeline
modelfile = 'models/udpipe_syntagrus.model'

def tag_ud(text, modelfile='udpipe_syntagrus.model'):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    processed = pipeline.process(text)
    output = [l for l in processed.split('\n') if not l.startswith('#')]
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in output if w]
    tagged_propn = []
    propn  = []
    for t in tagged:
        if t.endswith('PROPN'):
            if propn:
                propn.append(t)
            else:
                propn = [t]
        else:
            if len(propn) > 1:
                name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
                tagged_propn.append(name)
            elif len(propn) == 1:
                tagged_propn.append(propn[0])
            tagged_propn.append(t)
            propn = []
    return tagged_propn

In [ ]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

ft_model = gensim.models.fasttext.FastText.load('models/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')
# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("models/word2vec/ruscorpora_upos_skipgram_300_5_2018.vec", binary=False)

In [ ]:
# doc2vec for every news title
vec_dim = 300

def create_average_vec(doc):
    average = np.zeros((vec_dim,), dtype='float32')
    num_words = 0.
    for word in doc:
        if word in ft_model.wv.vocab:
            average = np.add(average, ft_model[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average


def create_doc2vec(text):
    text = str(text)
    processed_text = clean_text(text)
#     processed_ud = tag_ud(text=processed_text, modelfile=modelfile)
    vec = create_average_vec(processed_text)
    return vec

urls['doc2vec'] = urls['title'].apply(create_doc2vec)
urls.to_csv('data/news/urls_with_unique_titles_fasttext.csv')

In [1]:
# USEFUL CODE STARTS HERE
import numpy as np
import pandas as pd

from datetime import datetime
from datetime import timezone
from zipfile import ZipFile
import os

In [2]:
urls = pd.read_csv('data/news/urls_with_unique_titles_fasttext.csv', index_col=0)

In [3]:
def make_df(start_time, end_time):
    timestamps = sorted(os.listdir('data/news/timestamps'))
    start_timestamp, end_timestamp = make_timestamps_from_datetime(start_time, end_time, timestamps)
    interval = make_interval(start_timestamp, end_timestamp, timestamps)
    with ZipFile('data/news/timestamps.zip') as timestamps_zip:
        df_list = [pd.read_csv(timestamps_zip.open("timestamps/" + file), header=None, names=['fullVisitorId', 'url_id', 'visitStartTime']) for file in interval]
    df = pd.concat(df_list)
    labels, levels = pd.factorize(df['fullVisitorId'])
    df['user_id'] = labels
    return df


# first = 12/03/2017 07:00:00(1491818423), last = 14/04/2017(1491991225) 11:11:29
def make_timestamps_from_datetime(start_time, end_time, timestamps):
    if start_time == 'first':
        start_timestamp = timestamps[0]
    else:
        start_datetime = datetime.strptime(start_time, '%d/%m/%Y %H:%M:%S')
        start_timestamp = (start_datetime - datetime(1970, 1, 1)).total_seconds()
        
    if end_time == 'last':
        end_timestamp = timestamps[-1]
    else:
        end_datetime = datetime.strptime(end_time, '%d/%m/%Y %H:%M:%S')
        end_timestamp = (end_datetime - datetime(1970, 1, 1)).total_seconds()
    return (start_timestamp, end_timestamp)


def make_interval(start_timestamp, end_timestamp, timestamps):
    start_timestamp = str(start_timestamp)
    end_timestamp = str(end_timestamp)
    interval = [t for t in timestamps if t >= start_timestamp and t <= end_timestamp]
    return interval


def merge_df(df, urls):
    urls['url_id'] = urls['url_id'].astype(int)
    urls['title'] = urls['title'].astype(str)
    
    df_result = pd.merge(df, urls, on='url_id', how='left')

    labels, levels = pd.factorize(df_result['url_id'])
    df_result['url_id'] = labels
    df_result.set_index(['user_id', 'url_id'], inplace=True)
    df_result.sort_index(inplace=True)
    df_result.dropna(how='any',inplace=True)
    df_result.drop_duplicates(inplace=True)
    
    df_result['fullVisitorId'] = df_result['fullVisitorId'].astype(str)
    df_result['title'] = df_result['title'].astype(str)
    df_result['pagePath'] = df_result['title'].astype(str)

    return df_result

In [4]:
# You can choose time interval beetween 12/03/2017 07:00:00 and 14/04/2017 11:11:29
# Also you can use as arguments 'first' or 'last'
df = make_df('12/04/2017 10:00:00', '12/04/2017 11:00:00')
df_result = merge_df(df, urls)

In [5]:
# Calculates user clicks score for each news article
scores = df_result['pagePath'].value_counts()
df_result['scores'] = df_result['pagePath'].apply(lambda path: scores[str(path)])

In [6]:
display(df_result.info())
display(df_result.head(20))
display(df_result.tail(20))

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 356277 entries, (0, 0) to (108635, 723)
Data columns (total 6 columns):
fullVisitorId     356277 non-null object
visitStartTime    356277 non-null int64
title             356277 non-null object
pagePath          356277 non-null object
doc2vec           356277 non-null object
scores            356277 non-null int64
dtypes: int64(2), object(4)
memory usage: 19.2+ MB


None

fullVisitorId  visitStartTime  \
user_id url_id                                         
0       0        6645641729271449996      1491991166   
        0        6645641729271449996      1491991225   
        107      6645641729271449996      1491991270   
        107      6645641729271449996      1491991316   
        242      6645641729271449996      1491991323   
        242      6645641729271449996      1491991345   
        596      6645641729271449996      1491991323   
        598      6645641729271449996      1491991270   
1       1        1936127266341456044      1491991208   
        239      1936127266341456044      1491991198   
2       2        5068230910030454472      1491991164   
        92       5068230910030454472      1491991164   
3       3       -8872234143066009107      1491991215   
        8       -8872234143066009107      1491991705   
        19      -8872234143066009107      1491991730   
        28      -8872234143066009107      1491991896   
        29      -8872234143066009107      1491991953   
        58      -8872234143066009107      1491991751   
        79      -8872234143066009107      1491991761   
        79      -8872234143066009107      1491991766   

                                                            title  \
user_id url_id                                                      
0       0       Британская певица загорает на пляже голой на г...   
        0       Британская певица загорает на пляже голой на г...   
        107     Минкультуры Украины осудило слова Дорна о брат...   
        107     Минкультуры Украины осудило слова Дорна о брат...   
        242     "Ла-ла-ла-ла-ла, всё будет хорошо". В Сети опу...   
        242     "Ла-ла-ла-ла-ла, всё будет хорошо". В Сети опу...   
        596     "Хочешь, я буду твоей первой женщиной?" Серябк...   
        598     Директор Дорна о его скандальном интервью: Ива...   
1       1       Лесби-поцелуи у Кремля и травести-шоу. Как отр...   
        239     Дагестанский чиновник потратил миллионы на сва...   
2       2       СМИ: 19 апреля астероид пролетит в двух миллио...   
        92              Студенческий стартап: от идеи до продукта   
3       3       Глава ФСБ призвал усилить контроль на госграни...   
        8       В Дербенте спецслужбы обезвредили пять самодел...   
        19         Путин о действиях США в Сирии: Скучно, девочки   
        28      Члены СБ ООН потребовали от властей Сирии пред...   
        29      СМИ пишут о замене ворот, через которые лондон...   
        58      Кличко предложил переименовать проспект Победы...   
        79      СМИ: Тереза Мэй побоялась отпускать Джонсона н...   
        79      СМИ: Тереза Мэй побоялась отпускать Джонсона н...   

                                                         pagePath  \
user_id url_id                                                      
0       0       Британская певица загорает на пляже голой на г...   
        0       Британская певица загорает на пляже голой на г...   
        107     Минкультуры Украины осудило слова Дорна о брат...   
        107     Минкультуры Украины осудило слова Дорна о брат...   
        242     "Ла-ла-ла-ла-ла, всё будет хорошо". В Сети опу...   
        242     "Ла-ла-ла-ла-ла, всё будет хорошо". В Сети опу...   
        596     "Хочешь, я буду твоей первой женщиной?" Серябк...   
        598     Директор Дорна о его скандальном интервью: Ива...   
1       1       Лесби-поцелуи у Кремля и травести-шоу. Как отр...   
        239     Дагестанский чиновник потратил миллионы на сва...   
2       2       СМИ: 19 апреля астероид пролетит в двух миллио...   
        92              Студенческий стартап: от идеи до продукта   
3       3       Глава ФСБ призвал усилить контроль на госграни...   
        8       В Дербенте спецслужбы обезвредили пять самодел...   
        19         Путин о действиях США в Сирии: Скучно, девочки   
        28      Члены СБ ООН потребовали от властей Сирии пред...   
        29      СМИ пишут о з

,,fullVisitorId,visitStartTime,title,pagePath,doc2vec,scores
user_id,url_id,,,,,,
108616,709,-8551171097793120250,1491994720,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108617,709,4233325867084343019,1491994749,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108618,709,-5622289791728482319,1491994717,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108619,709,-2092817677204950029,1491994729,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108620,709,1927197122805736801,1491994761,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108621,709,3121942285589557985,1491994749,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108622,709,-9008640716598224968,1491994749,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108623,709,5343221902902809533,1491994733,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
108624,709,2779683125797673419,1491994743,Конгрессмен обсудит с директором ФБР арест Соб...,Конгрессмен обсудит с директором ФБР арест Соб...,[ 3.00536789e-02 3.50083224e-03 3.4118935...,7906
